In [ ]:
#Constraints For The Given Problem

"""
Hard Constraints (10 Points Each)
• An exam will be scheduled for each course.
• A student is enrolled in at least 3 courses. A student cannot give more than 1 exam at a time.
• Exam will not be held on weekends.
• Each exam must be held between 9 am and 5 pm
• Each exam must be invigilated by a teacher. A teacher cannot invigilate two exams at the same time.
• A teacher cannot invigilate two exams in a row.

Soft Constraints (5 Points Each)
• All students and teachers shall be given a break on Friday from 1-2.
• A student shall not give more than 1 exam consecutively.
• Two hours of break in the week such that at least half the faculty is free in one slot and the rest of the faculty is free in the other slot so the faculty meetings shall be held in parts as they are now.
• If a student is enrolled in a MG course and a CS course, it is preferred that their MG course exam be held before their CS course exam.
Additional Constraints (Mentioned Later on, Not sure about the points)
• Use binary encoding for chromosome.

Additional Constraints (Mentioned Later on, Points not known)
• 28 Students per Class

"""

#Deliverables

"""
.pynb File
One Page PDF Report
"""


In [1]:
# %%

import csv
import math

In [2]:
#Data Extraction from Files
def readStudentData(filename):
    #Storing student data in dictionary
    #Key is 0,1,2,3 ; value is student name
    studentData = dict()
    count = 0
    with open(filename,mode = 'r') as studentdatafile:
        read = csv.reader(studentdatafile,delimiter = ',')
        for name in read:
            #studentData.add(name[0])
            studentData[count] = name[0]
            count += 1
    #print(studentData)
    return studentData
def readTeacherData(filename):
    #Storing teacher data in dictionary
    #key is 0,1,2,3 ; value is teacher names
    teacherData = dict()
    count = 0
    with open(filename,mode = 'r') as teacherdatafile:
        read = csv.reader(teacherdatafile,delimiter = ',')
        for name in read:
            if len(name) == 0:
                continue
            teacherData[count] = name[0]
            count += 1
    #print(teacherData)
    return teacherData
def readCourseData(filename):
    #Storing Course data
    #Key is course code, value is course name
    coursesData = dict()
    count = 0
    with open(filename,mode = 'r') as coursesdatafile:
        read = csv.reader(coursesdatafile,delimiter = ',')
        for name in read:
            coursesData[name[0]] = name[1]
    #print(coursesData)
    return coursesData
def readstudentInCourseData(filename, coursesData):
    #Storing student in each course
    #Key is course; value is list of studen taking course
    studentInCourseData = dict()
    for i in coursesData:
        studentInCourseData[i] = []
    count = 0
    with open(filename, mode = 'r') as student_in_course_datafile:
        read = csv.reader(student_in_course_datafile, delimiter = ',')
        for name in read:
            #Headers in studentCourse.csv
            if ( count == 0 ):
                count += 1
                continue
            studentInCourseData[name[2]].append(name[1])
            count += 1
    #print(studentInCourseData)
    return studentInCourseData
def coursesToStudentData(filename, studentData):
    #Storing courses taken by each student
    #Key is student; value is list of courses taken by student
    coursesToStudentData = dict()
    for i in studentData:
        coursesToStudentData[studentData[i]] = []
    count = 0
    with open(filename, mode = 'r') as courses_to_student_datafile:
        read = csv.reader(courses_to_student_datafile, delimiter = ',')
        for name in read:
            #Headers in studentCourse.csv
            if ( count == 0 ):
                count += 1
                continue
            coursesToStudentData[name[1]].append(name[2])
            count += 1
    #print(coursesToStudentData)
    return coursesToStudentData

In [3]:
#Miscellaneous Functions
def avg_students_per_course(studentInCourseData):
    sumStudents = 0
    leng = 0
    for i in studentInCourseData:
        sumStudents += len(studentInCourseData[i])
        leng += 1
    return (sumStudents / leng)

In [9]:
if __name__ == "__main__":
    student_data = readStudentData('studentNames.csv')
    teacher_data = readTeacherData('teachers.csv')
    course_data = readCourseData('courses.csv')
    student_in_course = readstudentInCourseData('studentCourse.csv', course_data)
    courses_to_student = coursesToStudentData('studentCourse.csv', student_data)
    print(f"Number of Students: {len(student_data.keys())}")
    print(f"Number of Teachers: {len(teacher_data.keys())}")
    print(f"Number of Courses: {len(course_data.keys())}")
    #print(f"Number of Students: {len(studentInCourseData.keys())}")
    #print(f"Number of Students: {len(CoursesToStudentData.keys())}")
    
    avg_students = avg_students_per_course(student_in_course)
    classrooms_per_course = math.ceil(len(teacher_data.keys())/len(course_data.keys()))
    avg_students_per_classroom = math.ceil(avg_students/classrooms_per_course)
    
    print(f"Average Students Per Course: {avg_students}")
    print(f"Classrooms Per Course: {classrooms_per_course}")
    print(f"Average Students Per Classroom: {avg_students_per_classroom}")
    
    
    
    

Number of Students: 200
Number of Teachers: 63
Number of Courses: 23
Average Students Per Course: 35.65217391304348
Classrooms Per Course: 3
Average Students Per Classroom: 12
